In [54]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
%matplotlib inline
mpl.style.use('ggplot')

In [55]:
##IMPORTED LIBRARIES ->READING DATA

In [56]:
df = pd.read_csv("quikr_car.csv")

In [57]:
##VIEWING DATA

In [58]:
df.head()

,name,company,year,Price,kms_driven,fuel_type
0,Hyundai Santro Xing XO eRLX Euro III,Hyundai,2007,"80,000","45,000 kms",Petrol
1,Mahindra Jeep CL550 MDI,Mahindra,2006,"4,25,000",40 kms,Diesel
2,Maruti Suzuki Alto 800 Vxi,Maruti,2018,Ask For Price,"22,000 kms",Petrol
3,Hyundai Grand i10 Magna 1.2 Kappa VTVT,Hyundai,2014,"3,25,000","28,000 kms",Petrol
4,Ford EcoSport Titanium 1.5L TDCi,Ford,2014,"5,75,000","36,000 kms",Diesel


In [59]:
df.shape

(892, 6)

In [60]:
##QUALITY Problems -> 
##checking each column by df['col_name'].unique()

In [61]:
##year has many garbage 
##values which are non-year

In [62]:
##Price has garbage values as well 
##convert to int and remove commas

##kms_driven should have int it has strings with 
##commas and kms at end aswell

In [63]:
##fuel_type has nan values

In [64]:
##Name is very inconsistent so we keep the 
##first three words only

In [65]:
backup = df.copy()

In [66]:
# DATA CLEANING

In [67]:
df = df[df['year'].str.isnumeric()]

In [68]:
df['year'] = df['year'].astype(int)

In [69]:
df=df[df['Price']!='Ask For Price']
df['Price']=df['Price'].str.replace(',','').astype(int)

In [70]:
df['kms_driven']=df['kms_driven'].str.split().str.get(0).str.replace(',','')

In [71]:
df=df[df['kms_driven'].str.isnumeric()]

In [72]:
df['kms_driven']=df['kms_driven'].astype(int)

In [73]:
df=df[~df['fuel_type'].isna()]

In [74]:
df['name']=df['name'].str.split().str.slice(start=0,stop=3).str.join(' ')

In [75]:
df=df.reset_index(drop=True)

In [76]:
df=df[df['Price']<6000000]

In [77]:
df.to_csv('Cleaned_Car_data.csv')

In [78]:
##SPLITTING THE DATA

In [79]:
X=df[['name','company','year','kms_driven','fuel_type']]
y=df['Price']

In [80]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2)

In [81]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.metrics import r2_score

In [82]:

##One-hot encoding is often used in linear regression 
##(and other machine learning algorithms) when dealing 
##with categorical variables. Linear regression works well 
##with numerical data, but it struggles with categorical 
##variables because it assumes a linear relationship between 
##the input features and the output variable."""

"""For example, if you have a categorical variable 
"color" with three categories: red, green, and blue, 
one-hot encoding would represent these categories as 
three binary features: is_red, is_green, and is_blue. 
Each feature would take the value of 0 or 1, indicating 
the absence or presence of that category for a particular 
data point."""

'For example, if you have a categorical variable \n"color" with three categories: red, green, and blue, \none-hot encoding would represent these categories as \nthree binary features: is_red, is_green, and is_blue. \nEach feature would take the value of 0 or 1, indicating \nthe absence or presence of that category for a particular \ndata point.'

In [147]:
# Define the column transformer
column_trans = make_column_transformer(
    (OneHotEncoder(), ['name', 'company', 'fuel_type']),
    remainder='passthrough'
)

In [148]:
lr=LinearRegression()

In [149]:
pipe=make_pipeline(column_trans,lr)

In [150]:
pipe.fit(X_train,y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(),
                                                  ['name', 'company',
                                                   'fuel_type'])])),
                ('linearregression', LinearRegression())])

In [151]:
# Save the trained model
pickle.dump(pipe, open('LinearRegressionModel.pkl', 'wb'))

In [152]:
r2_score(y_test,y_pred)

-1.7336762819170266

In [153]:
# Get the feature names
feature_names = column_trans.get_feature_names_out().tolist()

# Append additional features
expected_columns = feature_names + ['year', 'kms_driven']

# Save the list of expected columns
pickle.dump(expected_columns, open('Expected_Columns.pkl', 'wb'))

In [154]:
pipe.predict(pd.DataFrame(columns=X_test.columns,data=np.array(['Maruti Suzuki Swift','Maruti',2019,100,'Petrol']).reshape(1,5)))

array([456495.49614994])

In [155]:
import pickle

In [156]:
pickle.dump(pipe,open('LinearRegressionModel.pkl','wb'))

In [157]:
pipe.predict(pd.DataFrame(columns=['name','company','year','kms_driven','fuel_type'],data=np.array(['Maruti Suzuki Swift','Maruti',2019,100,'Petrol']).reshape(1,5)))

array([456495.49614994])

In [145]:
import sklearn

In [146]:
print('The scikit-learn version is {}.'.format(sklearn.__version__))

The scikit-learn version is 1.3.0.
